In [5]:
%%time
import segmind.instance as instance
import requests
import traceback
from datetime import datetime

try:
    import numpy as np
    from qutip import mesolve, basis, jmat

    def drive(t, args):
        w = args['omega']
        h = args['h']
        h0 = args['h0']
        return h * np.cos(w*t) + h0

    def get_hamiltonians(N):
        sx,sy,sz = jmat(N,"x"),jmat(N,"y"),jmat(N,"z")
        kn =  2.0/N                                      # kacNorm
        H0 = kn * sz **2 
        H1 = 2 * sx
        return H0,H1

    def floq_evolv(args):
        N = args['N']
        T = 2 * np.pi/args['omega']
        opts = args['opts']
        H0, H1 = get_hamiltonians(N)
        H = [H0,[H1,drive]]

        f_mat = []
        for i in range(int(2*N+1)):
            baseket = basis(int(2*N+1),i)
            solv_state = mesolve(H, baseket, [0,T], [], [], args = args, options=opts)
            f_mat.append(np.array(solv_state.states)[-1])

        return np.array(f_mat)[:,:,0]
    
    from qutip import Qobj, jmat, Options
    import numpy as np
    from multiprocessing import Pool
    from tqdm import tqdm
    from scipy.special import jn_zeros

    freezing_pts = jn_zeros(0,3)
    ####################################################################################################
    #
    #                                     Exact Dynamics
    #
    ####################################################################################################


    N = 7
    h0 = 0.0
    omega = 90
    nprocs = 13

    w = omega

    hsteps = 100
    #h1 = (1/4) * omega * 0.01
    h1 = 0
    h2 = (1/4) * omega * 16.0 

    hroots = freezing_pts * omega/4

    hs = np.linspace(h1, h2, hsteps)

    hs = np.sort(np.append(hs, hroots))

    p = Pool(processes = nprocs)          
    print("running for N = ", N, "with",nprocs,"processors")  

    #Qutip Solver Options
    opts = Options(nsteps=1e5, num_cpus=1, openmp_threads=1)

    params = [{'h0':0, 'h':h, 'omega':omega, 'N':N, 'opts':opts} for h in hs]
    evec_f = p.map(floq_evolv,tqdm(params, position=0, leave=True))


    iprmat = np.zeros((len(hs),2*N+1))

    sx = jmat(N,"x")
    en, st = sx.eigenstates()

    print("\n Done. Now calculating PR ...")
    for xx,h in enumerate(hs):
        floquet_matrix = Qobj(evec_f[xx]).transform(st)
        iprmat[xx,:] = np.sum(np.abs(np.array(floquet_matrix))**4, axis=1)


    print("Exact simulation complete! Saving Data ...")
    import h5py
    dt = f'{datetime.now():%m_%d_%Y_%H_%M_%S}'
    fname = "iprdata_" + dt + ".hdf5"
    with h5py.File(fname, 'w') as hf:
        ipr_dset =  hf.create_dataset("iprmat",np.shape(iprmat),  data=iprmat)
        ampl_dset = hf.create_dataset("amplitudes", np.shape(hs), data=hs)
        ampl_dset.attrs['omega'] = omega
        ampl_dset.attrs['N'] = N
    #################################################
    bot_message = "Code executed with no exceptions.\n"
except Exception:
    # This bit exists in case your code conks out with an exception
    # That way, the exception will no longer terminate your execution
    # without stopping the segmind instance
    print(traceback.format_exc())
    bot_message = f"Code exited with {Exception}. Check stderr for details.\n"

#####################################################
# You can customize the bot message as per your needs
#####################################################
whoami = "Analabha Roy"
# datetime object containing current date and time
now = datetime.now()
bot_message = 'Program run by ' + whoami + ' has been completed. Date/Time of termination:' + str(now) + bot_message
#####################################################

# The SegmindStatusBot is sent an instruction to broadcast the message in 'bot_message'
bot_token = 'PUT BOT TOKEN'
bot_chatID = 'PUT CHAT ID'
send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#response = requests.get(send_text)

from ipylab import JupyterFrontEnd
app = JupyterFrontEnd()

app.commands.execute('docmanager:save')

# The running instance is now automatically stopped so that it doesn't eat any more credits than it has to
#instance.stop()
print("Done!")

Definitions complete!
Running for N =  10 with 4 processors


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 32346.01it/s]



Done. Now calculating PR ...
Exact simulation complete! Saving Data ...
Done!
CPU times: user 29.3 ms, sys: 14.2 ms, total: 43.5 ms
Wall time: 12.8 s
